# Appending TD with `user_prompt`

In [ ]:
%pip install langchain-openai langchain-core langchain-community 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass.getpass("Enter OpenAI API Key: ")

In [2]:
from langchain_openai import ChatOpenAI

openai_model = "gpt-4.1"
# For normal accurate responses
llm = ChatOpenAI(temperature=0.0, model=openai_model)

In [ ]:
user_prompt = """
    Desired Workflow: \n {desired_workflow}
    \n\n
    List of Devices as WoT Thing Descriptions: \n {thing_directory}
    """

In [5]:
import sys
import os

parent_dir = os.path.abspath("..")
sys.path.append(parent_dir)

import utils


In [6]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate
)

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(utils.system_prompt),
    HumanMessagePromptTemplate.from_template(user_prompt)
])

In [7]:
prompt_template.input_variables

['desired_workflow', 'thing_directory']

In [8]:
# for testing purposes
print(prompt_template.format(desired_workflow="ipsum dolor", thing_directory="LOREM IPSUM"))

System: You are an expert IoT system developer, proficient with Web of Things (WoT) descriptions and Node-RED workflow programming. Ensure that all node IDs are unique. Ensure that quote marks used within strings are handled. Ensure your response is valid JSON. Your job is to take new IoT system proposals/descriptions (from users) along with a list of the devices that are available to use (as WoT Thing descriptions). From this information, you will produce an IoT system workflow, for use within Node-RED, which connects the relevant Things/devices in order to satisfy the requirements of the provided system proposal/description.
    Only output the generated workflow, no other explanation or text can be provided.
Human: 
    Desired Workflow: ipsum dolor
    List of Devices as WoT Thing Descriptions: 
 LOREM IPSUM
    


In [9]:
pipeline = (
    {
        "thing_directory": lambda x: x["thing_directory"],
        "desired_workflow": lambda x: x["desired_workflow"]
    }
    | prompt_template
    | llm
    | {"generated_workflow": lambda x: x.content}
)

In [10]:
response = pipeline.invoke({
    "thing_directory": utils.smart_home_td_full,
    "desired_workflow": utils.smart_home_system_description
})

In [ ]:
response



{'generated_workflow': '[\n  {\n    "id": "n1a1b1c1",\n    "type": "websocket in",\n    "z": "main",\n    "name": "WashingMachine finishedCycle",\n    "server": "",\n    "client": "",\n    "path": "ws://localhost:1880/things/wm/finishedCycle",\n    "x": 100,\n    "y": 60,\n    "wires": [\n      [\n        "n2a2b2c2"\n      ]\n    ]\n  },\n  {\n    "id": "n2a2b2c2",\n    "type": "http request",\n    "z": "main",\n    "name": "Blink LEDs",\n    "method": "POST",\n    "ret": "txt",\n    "url": "http://localhost:1880/things/leds/blink",\n    "headers": {},\n    "x": 320,\n    "y": 60,\n    "wires": [\n      []\n    ]\n  },\n  {\n    "id": "n3a3b3c3",\n    "type": "websocket in",\n    "z": "main",\n    "name": "MotionSensor motionDetected",\n    "server": "",\n    "client": "",\n    "path": "ws://localhost:1880/things/motionsensor/motiondetected",\n    "x": 100,\n    "y": 140,\n    "wires": [\n      [\n        "n4a4b4c4"\n      ]\n    ]\n  },\n  {\n    "id": "n4a4b4c4",\n    "type": "http r

In [12]:
# formatting the response

import json

# Parse twice: first to extract string, second to get proper JSON
workflow_str = response['generated_workflow']
workflow = json.loads(workflow_str)

# Now `workflow` is a list of dicts — ready for Node-RED
print(json.dumps(workflow, indent=2))

[
  {
    "id": "n1a1b1c1",
    "type": "websocket in",
    "z": "main",
    "name": "WashingMachine finishedCycle",
    "server": "",
    "client": "",
    "path": "ws://localhost:1880/things/wm/finishedCycle",
    "x": 100,
    "y": 60,
    "wires": [
      [
        "n2a2b2c2"
      ]
    ]
  },
  {
    "id": "n2a2b2c2",
    "type": "http request",
    "z": "main",
    "name": "Blink LEDs",
    "method": "POST",
    "ret": "txt",
    "url": "http://localhost:1880/things/leds/blink",
    "headers": {},
    "x": 320,
    "y": 60,
    "wires": [
      []
    ]
  },
  {
    "id": "n3a3b3c3",
    "type": "websocket in",
    "z": "main",
    "name": "MotionSensor motionDetected",
    "server": "",
    "client": "",
    "path": "ws://localhost:1880/things/motionsensor/motiondetected",
    "x": 100,
    "y": 140,
    "wires": [
      [
        "n4a4b4c4"
      ]
    ]
  },
  {
    "id": "n4a4b4c4",
    "type": "http request",
    "z": "main",
    "name": "Turn MainRoomLight On",
    "method